# Waterfall 

<p> The notebook calculates how many customers bought sequentially all subsequent seasons of a particular series </p>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../GOT/data/waterfall_4.txt", sep="\t",
                 names = ['encrypted_customer_id',
                          'transaction_date','asin',
                          'content_type','title'])

In [3]:
print(df.shape)
df.head()

(1229481, 5)


,encrypted_customer_id,transaction_date,asin,content_type,title
0,A3PCW9WC0GTF2Q,2017-08-15 00:00:00,B073VVVKNB,TV Season,Game of Thrones - Staffel 7 [dt./OV]
1,A1A77UBUPOPZI1,2017-08-21 00:00:00,B00M3LG74C,TV Season,Game of Thrones - Staffel 4 [dt./OV]
2,A3GKVH6E99LI3V,2017-07-30 00:00:00,B00I8ZYTRO,TV Season,Game of Thrones - Staffel 1 [dt./OV]
3,A2R45O6WIV592C,2014-08-05 00:00:00,B00M3LG74C,TV Season,Game of Thrones - Staffel 4 [dt./OV]
4,A9E4FJS969OCG,2016-09-18 00:00:00,B00I8ZVHDI,TV Season,Game of Thrones - Staffel 3 [dt./OV]


In [4]:
def to_preprocess(df):
    df['season_number'] =    df.title.str.extract('(\d+)')
    df['season_number'] =    pd.to_numeric(df['season_number'] , errors='coerce')

    df['transaction_date'] = pd.to_datetime(df['transaction_date'])
    df['days_since_epoch'] = (pd.to_datetime(df['transaction_date']) - pd.datetime(1970,1,1)).dt.days
    return df
    
new_df = to_preprocess(df)

C:\Users\dfadeeff\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [5]:
print("Overall number of unique customers who bought: " , new_df['encrypted_customer_id'].nunique())
print("Unique asins: ", new_df['asin'].nunique())

Overall number of unique customers who bought:  515182
Unique asins:  14


In [6]:
def recursive_call_funnel(df,staffel, base):
    """calculates customers who watched all previous seasons"""
    if staffel <= base:
        return df[df['season_number'] == base]['encrypted_customer_id']
    else :
        return df[(df['season_number'] == staffel) 
                  & (df['encrypted_customer_id'].isin(recursive_call_funnel(df,staffel-1,base)))]['encrypted_customer_id']   


In [ ]:
def create_matrix(df,range_start,range_end):
    
    new_table = []
    
    for i in range(range_start,range_end + 1):
        new_list = []
        for j in range(i, range_end + 1):
            new_list.append(recursive_call_funnel(df,j,i).nunique())
        new_table.append(new_list)
    return new_table

In [ ]:
new_table_total = create_matrix(new_df, 1, 8)